In [83]:
import pandas as pd
import pandasql as pdsql

In [84]:
filename = 'weather_underground.csv'
weather_data = pd.read_csv('./data/' + filename)

In [85]:
weather_data.describe()

,maxpressurem,maxdewptm,maxpressurei,maxdewpti,since1julheatingdegreedaysnormal,heatingdegreedaysnormal,since1sepcoolingdegreedaysnormal,hail,since1julsnowfallm,since1julheatingdegreedays,...,precipi,snowfalli,since1jancoolingdegreedaysnormal,precipm,snowfallm,thunder,monthtodateheatingdegreedays,meantempi,maxvism,meantempm
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,0.0,30.0,3.000000e+01,30.000000,...,30.000000,30.0,30.000000,30.000000,30.0,30.0,30.000000,30.000000,30.000000,30.000000
mean,1016.733333,14.066667,30.031333,57.266667,4717.700000,4.166667,NaN,0.0,1.572300e+02,4691.100000,...,0.172667,0.0,34.300000,4.329000,0.0,0.0,61.100000,64.300000,15.733333,17.900000
std,4.425111,4.940566,0.127840,8.920852,35.338901,1.821014,NaN,0.0,2.890759e-14,28.807746,...,0.437051,0.0,17.186904,11.119453,0.0,0.0,28.807746,6.685806,1.142693,3.744882
min,1007.000000,4.000000,29.740000,39.000000,4646.000000,2.000000,NaN,0.0,1.572300e+02,4635.000000,...,0.000000,0.0,13.000000,0.000000,0.0,0.0,5.000000,55.000000,10.000000,13.000000
25%,1014.250000,10.000000,29.965000,50.000000,4692.250000,3.000000,NaN,0.0,1.572300e+02,4669.500000,...,0.000000,0.0,20.250000,0.000000,0.0,0.0,39.500000,60.250000,16.000000,16.000000
50%,1017.000000,14.500000,30.030000,58.000000,4725.000000,4.000000,NaN,0.0,1.572300e+02,4690.500000,...,0.000000,0.0,30.000000,0.000000,0.0,0.0,60.500000,63.000000,16.000000,17.000000
75%,1019.000000,17.750000,30.097500,63.750000,4748.250000,5.750000,NaN,0.0,1.572300e+02,4723.000000,...,0.092500,0.0,46.250000,2.350000,0.0,0.0,93.000000,67.750000,16.000000,19.750000
max,1026.000000,21.000000,30.310000,70.000000,4763.000000,8.000000,NaN,0.0,1.572300e+02,4724.000000,...,2.180000,0.0,71.000000,55.370000,0.0,0.0,94.000000,78.000000,16.000000,26.000000


#### Q1: Find the number of days when it rained

In [86]:
def num_rainy_days(weather_data):
    
    q = """
    SELECT count(*)
    FROM weather_data wd
    WHERE wd.rain == 1
    """
    
    rainy_days = pdsql.sqldf(q.lower(), locals())
    
    return rainy_days

In [87]:
num_rainy_days(weather_data)

,count(*)
0,10


#### Q2: The SQL query should return two columns and two rows - whether it was foggy or not (0 or 1) and the max maxtempi for that fog value (i.e., the maximum max temperature for both foggy and non-foggy days).

In [88]:
def max_temp_aggregate_by_fog(weather_data):
    
    q = """
    SELECT wd.fog, cast(max(maxtempi) as integer)
    FROM weather_data wd
    GROUP BY wd.fog
    """
    
    foggy_days = pdsql.sqldf(q.lower(), locals())
                       
    return foggy_days

In [89]:
max_temp_aggregate_by_fog(weather_data)

,fog,cast(max(maxtempi) as integer)
0,0,86
1,1,81


#### 3. The SQL query should return one column and one row - the average meantempi on days that are a Saturday or Sunday (i.e., the the average mean temperature on weekends).

In [90]:
def avg_weekend_temperature(weather_data):
    
    q = """
    SELECT AVG(meantempi)
    FROM weather_data
    WHERE cast(strftime('%w', date) as integer) == 0 OR cast(strftime('%w', date) as integer) == 6
    """
    
    mean_temp_weekends = pdsql.sqldf(q.lower(), locals())
    
    return mean_temp_weekends

In [91]:
avg_weekend_temperature(weather_data)

,avg(meantempi)
0,65.111111


#### 4. You want to find the average minimum temperature (mintempi column of the weather dataframe) on rainy days where the minimum temperature is greater than 55 degrees.

In [92]:
def avg_min_temperaure(weather_data):
    
    q= """
    SELECT AVG(mintempi)
    FROM weather_data
    WHERE rain >= 1
        AND mintempi > 55
    """
    
    avg_min_temp_rainy = pdsql.sqldf(q.lower(), locals())
    return avg_min_temp_rainy


In [93]:
avg_min_temperaure(weather_data)

,avg(mintempi)
0,61.25


#### 5. Write the updates to a different text file in the format of "updated_" + filename.
    For example:
        1) if you read in a text file called "turnstile_110521.txt"
        2) you should write the updated data to "updated_turnstile_110521.txt"

In [94]:
import csv
test_file = './data/sample-mta-file.csv'
name = test_file

In [95]:
#for name in filenames:
        
f_in = open(name, 'r')
#new_fname = 'updated_' + name
new_fname = 'updated_sample-mta-file.csv'
f_out = open(new_fname, 'w')

reader_in = csv.reader(f_in, delimiter = ',')
writer_out = csv.writer(f_out, delimiter = ',')   

In [96]:
for line in reader_in:
            elements = len(line)
            num_lines = (elements - 3) / 5
            for i in range(num_lines):
                line_out = line[0], line[1], line[2], line[3+5*i], line[4+5*i], line[5+5*i], line[6+5*i], line[7+5*i]
                writer_out.writerow(line_out)

TypeError: 'float' object cannot be interpreted as an integer

In [97]:
f_in.close()
f_out.close() 

####  6. Write a function that takes the files in the list filenames, which all have the  columns 'C/A, UNIT, SCP, DATEn, TIMEn, DESCn, ENTRIESn, EXITSn', and consolidates them into one file located at output_file.  There should be ONE row with the column headers, located at the top of the file. The input files do not have column header rows of their own.

In [98]:
'''with open(output_file, 'w') as master_file:
        master_file.write('C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn\n')
        for filename in filenames:
            with open(filename) as infile:
                for line in infile:
                    master_file.write(line)
'''

"with open(output_file, 'w') as master_file:\n        master_file.write('C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn\n')\n        for filename in filenames:\n            with open(filename) as infile:\n                for line in infile:\n                    master_file.write(line)\n"

#### 7. This function should read the csv file located at filename into a pandas dataframe, and filter the dataframe to only rows where the 'DESCn' column has the value 'REGULAR'.

In [99]:
'''
df = pandas.read_csv(filename)
turnstile_data = df[df['DESCn'] == 'REGULAR']
'''

"\ndf = pandas.read_csv(filename)\nturnstile_data = df[df['DESCn'] == 'REGULAR']\n"

#### 8. and 9. More specifically, you want to do two things: 
1) Create a new column called ENTRIESn_hourly 
2) Assign to the column the difference between ENTRIESn of the current row and the previous row. If there is any NaN, fill/replace it with 1.

In [100]:
#df['ENTRIESn_hourly'] = df['ENTRIESn'] - df['ENTRIESn'].shift(1)
#df['ENTRIESn_hourly'] = df['ENTRIESn_hourly'].fillna(1)

#df['EXITSn_hourly'] = df['EXITSn'] - df['EXITSn'].shift(1)
#df['EXITSn_hourly'] = df['EXITSn_hourly'].fillna(0)

#hour = int(time[:2])

In [101]:
time = "00:00:00"

#### 11. In order to join these two data sets together, we'll want the dates formatted the same way.  Write a function that takes as its input a date in the MTA Subway data format, and returns a date in the weather underground format.

In [108]:
#date_formatted = datetime.datetime.strptime(date, "%m-%d-%y").strftime('%Y-%m-%d')